# Author: Tobias

This is the main notebook we use for model training. Is it mainly used to instantiate a benchmark, run the training, do the evaluation and save the model as well as the results.

First, we do the usual setup.

In [ ]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

In [5]:
import os 
import tensorflow as tf
from pathlib import Path

In [6]:
import cv2

In [ ]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)


In [8]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight

In [ ]:
!remote_access/get_tunnels.sh

We specify subsets of the columns we want to train on as well as different uncertainty encodings and transformations that are then handed to the data generator and applied before training.

In [19]:
columns_12 = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']

uzeros = ['Cardiomegaly',
        'Enlarged Cardiomediastinum',
        'Lung Opacity',
        'Lung Lesion',
        'Consolidation',
        'Pneumothorax',
        'Pleural Effusion']

uones = ['Edema',
        'Atelectasis',
        'Fracture',
        'Pleural Other',
        'Pneumonia',]


upsample_factors = {
    "Enlarged Cardiomediastinum": 1,
    "Lung Lesion":1,
    #"Pneumothorax":1,
    #"Pneumonia":1,
    "Pleural Other":2,
    "Fracture":2,
}

columns_5 =  ['Cardiomegaly',
                'Edema',
                'Consolidation',
                'Atelectasis',
                'Pleural Effusion']

uzeros_5 = ['Cardiomegaly',
        'Consolidation',
        'Pleural Effusion']

uones_5 = ['Edema',
        'Atelectasis']


upsample_factors_5 = {
    "Consolidation":2,
    "Cardiomegaly":1
}

transformations_0 = {"hist_equalization":{}}
transformations_1 = { "gaussian_blur":{"kernal_size":3}, "hist_equalization":{}}
transformations_2 = {"unsharp_mask":{"radius":2, "amount":1}}
transformations_3 = {"windowing"}

In [ ]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                 name="Chexpert_BCE_E3_B32_C0_N12_AugAffine_Uones_D256_DS9505_2LR1_LF5_SGD_Upsampled",
                                 classes=columns_12,
                                 train_labels = "train.csv",
                                 test_labels = "test.csv",
                                 nan_replacement = 0, #float("NaN"),
                                 u_enc = "uones",
                                 epochs=3,
                                 batch_size=32,
                                 crop = False,
                                 dim=(256, 256),
                                 loss = BinaryCrossentropy(),
                                 use_class_weights = False,
                                 upsample_factors = upsample_factors,
                                 metrics=METRICS,
                                 single_class_metrics=SINGLE_CLASS_METRICS,
                                 optimizer = SGD(learning_rate=2e-1, clipnorm=1),
                                 lr_factor = 0.5,
                                 augmentation = "affine",
                                 transformations = {},
                                 split_seed = 6122156,
                                 split_valid_size = 0.05, 
                                 preprocess_input_fn = tf.keras.applications.densenet.preprocess_input)

#bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
 #                                               bce_benchmark.negative_weights)

In [ ]:
bce_benchmark.as_dict()

In [21]:
bce_chexpert_exp = simple_architecture_experiment(bce_benchmark, DenseNet121, bce_benchmark.label_columns)

In [ ]:
bce_chexpert_exp.train()

In [ ]:
bce_chexpert_exp.evaluate()

In [ ]:
bce_chexpert_exp.save()

We can use this benchmark to verify various aspects of the training.

In [ ]:
compute_class_weight(bce_chexpert_exp.benchmark.traingen)

In [ ]:
trained_weights = bce_chexpert_exp.model.get_weights()

In [ ]:
images, labels = bce_chexpert_exp.benchmark.traingen[0]
len(images)

In [ ]:
preds = bce_chexpert_exp.model(images, training=True)
preds.op

In [ ]:
bce_chexpert_exp.model.summary(line_length=150)

In [ ]:
index = -1
trained_weights[index].shape, trained_weights[index], np.isnan(trained_weights[index]).any()

In [ ]:
nan_layers = [i for i in range(len(trained_weights)) if np.isnan(trained_weights[i]).any()] 
#nan_layers   

In [ ]:
mean_weights = {i:np.abs(trained_weights[i]).mean() for i in range(len(trained_weights))}
#mean_weights

In [ ]:
{i:trained_weights[i].shape for i in range(len(trained_weights))}

In [ ]:
for i in range(191, 195):
    print(i, trained_weights[i].shape, trained_weights[i])

In [ ]:
#chexpert_exp.save()

In [ ]:
from datetime import datetime
model_filename = chexpert_exp.model_name + "_" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_filename_tf = model_filename + ".tf"
model_filename_h5 = model_filename + ".h5"
model_filename

In [ ]:
folderpath = Path(os.getcwd()) / 'models' / chexpert_exp.model_name
path = folderpath / model_filename
path

In [ ]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(path, save_format="tf")

In [ ]:
# make sure path exists, ceate one if necessary
Path(folderpath).mkdir(parents=True, exist_ok=True)
chexpert_exp.model.save(folderpath / model_filename_h5, save_format="h5")

In [ ]:
!pwd

In [ ]:
folderpath / model_filename_h5

In [ ]:
chexpert_exp.save()

In [ ]:
chexpert_exp.evaluate()

In [ ]:
testgen = chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].testgen

In [ ]:
preds = chexpert_exp.model.predict(testgen, steps=len(testgen), verbose=1)

In [ ]:
report = classification_report(groundtruth_label, y_pred, target_names=chexpert_benchmarks["BCE_E1_B32_C0_N5_D256_DS0595_savetest7"].label_columns)
print('sklearn report: ', report)

We can also use the benchmark class to evaluate models other than the base models.

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)